# Data preprocessing 

In [84]:
from __future__ import print_function
from sklearn.preprocessing import Normalizer
import random
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader,random_split,TensorDataset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import confusion_matrix, f1_score, ConfusionMatrixDisplay
from sklearn import preprocessing
import time
from obspy.clients.fdsn import Client
from obspy.core import UTCDateTime
from obspy import read

# read data

In [85]:
# read data 
from glob import glob
file_list = glob("*.mseed")
for i,file in enumerate(file_list):
    if i ==0:
        st = read(file)
    else:
        st += read(file)
print(st)

# get the total number of traces in st
num_of_traces= (len(st))
print(num_of_traces)

# 720 data points in a single trace
for trace in st:
    samples = len(trace)
#     print(samples)

4364 Trace(s) in Stream:

IU.ADK.00.LHZ | 2003-07-27T05:25:31.848146Z - 2003-07-27T07:25:21.848146Z | 0.1 Hz, 720 samples
...
(4362 other traces)
...
IU.YSS.00.LHZ | 2003-05-26T18:23:29.298340Z - 2003-05-26T20:23:19.298340Z | 0.1 Hz, 720 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]
4364


In [86]:
# st[30].plot();

In [87]:
# set timer
time_start = time.time()

# Dataset and Sliding window

In [88]:
class ApterStDataset(Dataset):
    def __init__(self,base_size = 360,size = 10,drop_last=True,mode="train"):
        #mode train,test,val
        self.st = np.vstack([st]) # st(2794,720)
        self.st = preprocessing.scale(self.st)
        if mode == "train":
            self.st = self.st[0:int(len(self.st)*0.8)] # train mode : use 0-0.8 as train data
        elif mode=="test":
            self.st = self.st[int(len(self.st)*0.8):int(len(self.st)*0.9)] # test mode: use 0.8-0.9 as test data
        elif mode=="val":
            self.st = self.st[int(len(self.st)*0.9):int(len(self.st))] # val mode: use 0.9-1.0 as test data
        else:
            raise "mode error"
        
        self.size = size # moving window size
        self.drop_last = drop_last # drop the last batch if the no. of data less than size
        self.all_feture = self.init_all_feture(size = size,drop_last=drop_last,base_size=base_size) 
    
    def init_all_feture(self,size,base_size,drop_last=True):
        all_feture = []
        for i in self.st:      
            for j in range(base_size,720+size,size):
                temp_st = i[j-base_size:j] # moving range
                if j <= 360:
                    label = np.array([0]) # non-seismic event labelling 
                else:
                    label = np.array([1]) # seismic event labelling 
                if drop_last:   
                    if len(temp_st) == base_size:
                        all_feture.append((temp_st,label))
                else:
                    all_feture.append((temp_st,label))
        return all_feture


    def __len__(self):
        return len(self.all_feture)

    def __getitem__(self, idx): 
        feture = torch.from_numpy(self.all_feture[idx][0])
        # feture = feture.resize(batch)
        feture = feture.unsqueeze(1)  # add 1 dimension -> [size,1]
        label = torch.from_numpy(self.all_feture[idx][1])
        label = label.to(torch.long) # to int64
        return feture,label

#ApterStDataset
# asd = ApterStDataset(size=10,drop_last=True,mode="val")


In [89]:
batch = 20
base_size = 180
size = 10

train_asd = ApterStDataset(base_size=base_size,size=size,drop_last=True,mode="train")
test_asd = ApterStDataset(base_size=base_size,size=size,drop_last=True,mode="test")
val_asd = ApterStDataset(base_size=base_size,size=size,drop_last=True,mode="val")

#DataLoader
train_dl = torch.utils.data.DataLoader(
    train_asd,
    batch_size=batch,
    shuffle = True, # shuffle the data
    drop_last = True, # drop last batch
    num_workers=0      
)
test_dl = torch.utils.data.DataLoader(
    test_asd,
    batch_size=batch,
    shuffle = True, # shuffle the data
    drop_last = True, # drop last batch
    num_workers=0     
)
val_dl = torch.utils.data.DataLoader(
    val_asd,
    batch_size=batch,
    shuffle = True, # shuffle the data
    drop_last = True, # drop last batch
    num_workers=0      
)
for i,data in enumerate(test_dl):
    print(data)
    break

[tensor([[[ 0.0175],
         [ 0.0089],
         [ 0.0053],
         ...,
         [ 0.0123],
         [ 0.0144],
         [ 0.0065]],

        [[ 0.0101],
         [ 0.0185],
         [ 0.0150],
         ...,
         [-0.0154],
         [-0.0193],
         [-0.0218]],

        [[-0.0306],
         [-0.0222],
         [-0.0134],
         ...,
         [-0.0181],
         [-0.0184],
         [-0.0140]],

        ...,

        [[-0.0101],
         [-0.0108],
         [-0.0215],
         ...,
         [ 0.0146],
         [ 0.0106],
         [ 0.0014]],

        [[-0.0207],
         [-0.0152],
         [-0.0119],
         ...,
         [-0.0079],
         [-0.0182],
         [-0.0305]],

        [[-0.0174],
         [-0.0167],
         [-0.0172],
         ...,
         [-0.0081],
         [ 0.0183],
         [ 0.0156]]], dtype=torch.float64), tensor([[0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
    

In [90]:
len(train_dl)

9600

In [91]:
len(test_dl)

1199

In [92]:
len(val_dl)

1201

# The 1D CNN model

In [93]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=base_size, out_channels=100, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=100, out_channels=50, kernel_size=1)

        self.fc1 = nn.Linear(in_features=50, out_features=40)
        self.fc2 = nn.Linear(in_features=40, out_features=30)
        self.out = nn.Linear(in_features=30, out_features=2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
       
        x = x.flatten(1) # flatten the tensor starting at dimension 1

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x

# if torch.cuda.is_available():
#     cnn_cuda=cnn_cuda.cuda()
    
if __name__=='__main__':
    cnn = Network()
    input = torch.ones(batch, base_size, 1)
    output = cnn(input)
    print(output.shape)

torch.Size([20, 2])


In [94]:
cnn.conv1.weight.shape

torch.Size([100, 180, 1])

In [95]:
cnn.conv2.weight.shape

torch.Size([50, 100, 1])

In [96]:
cnn.fc1.weight.shape

torch.Size([40, 50])

In [97]:
cnn.fc2.weight.shape

torch.Size([30, 40])

In [98]:
cnn.out.weight.shape

torch.Size([2, 30])

# Training and testing

In [99]:
# parameters for training model
# total No. of traing
total_train_step = 0

# total No. of testing
total_test_step = 0

# Loss function 
loss_fn = nn.CrossEntropyLoss()
cnn = Network()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()
    cnn = cnn.to("cuda")
    
# Optimizer
learning_rate = 0.0001
# optimizer = torch.optim.SGD(cnn.parameters(), lr = learning_rate, momentum=0.9)
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate, weight_decay=1e-5)

# epoch
epoch = 10


In [100]:
# cnn.load_state_dict(torch.load("best_network.pth"))

In [ ]:
train_losses = []
train_acces = []
test_losses = []
test_acces = []

for i in range(epoch):

    # training
    train_loss = 0.0
    train_acc = 0.0
    cnn.train()  # training mode

    for data in train_dl:
        train_x, train_y = data
        train_y = train_y.resize(batch)
        if torch.cuda.is_available():
            train_x = train_x.cuda()
            train_y = train_y.cuda()
        train_x = train_x.to(torch.float)
        outputs = cnn(train_x)          # use train_x as inputs to the model
        loss = loss_fn(outputs,train_y) # calculate the loss between outputs and train_y

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # record train loss
        train_loss += loss.item()

        # train accuracy
        train_accuracy = (outputs.argmax(1) == train_y).sum()
        train_acc += train_accuracy.cpu()

    train_losses.append(train_loss/len(train_asd))
    train_acces.append(train_acc/len(train_asd))

    # testing
    test_loss = 0.0
    test_acc = 0.0
    cnn.eval()

    with torch.no_grad():
        for data in test_dl:
            test_x,test_y = data
            test_y = test_y.resize(batch)
            if torch.cuda.is_available():
                test_x = test_x.cuda()
                test_y = test_y.cuda()
            test_x = test_x.to(torch.float)

            outputs = cnn(test_x)
            loss = loss_fn(outputs, test_y)

            # record train loss
            test_loss += loss.item()

            # test accuracy
            test_accuracy = (outputs.argmax(1) == test_y).sum()
            test_acc += test_accuracy.cpu()

        test_losses.append(test_loss/len(test_asd))
        test_acces.append(test_acc/len(test_asd))
    print("epoch:{},Train Loss: {:.4f},Train Acc: {:.2f},Test Loss: {:.4f},Test Acc: {:.2f}".format(i+1,
          train_loss/len(train_asd),
          train_acc/len(train_asd),
          test_loss/len(test_asd),
          test_acc/len(test_asd)))

    torch.save(cnn, 'best_cnn.pth')  # save the network
    torch.save(cnn.state_dict(), 'cnn_params.pth')   # only save parameters


#     # early stopping
#     early_stopping(test_loss, cnn)

#     if early_stopping.early_stop:
#         print("Early stopping")
#         break

/Users/liuxiaoyu/opt/anaconda3/envs/python3_7/lib/python3.7/site-packages/torch/_tensor.py:586: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


epoch:1,Train Loss: 0.0074,Train Acc: 0.94,Test Loss: 0.0016,Test Acc: 0.99
epoch:2,Train Loss: 0.0019,Train Acc: 0.99,Test Loss: 0.0015,Test Acc: 0.99
epoch:3,Train Loss: 0.0016,Train Acc: 0.99,Test Loss: 0.0015,Test Acc: 0.99
epoch:4,Train Loss: 0.0014,Train Acc: 0.99,Test Loss: 0.0014,Test Acc: 0.99
epoch:5,Train Loss: 0.0012,Train Acc: 0.99,Test Loss: 0.0013,Test Acc: 0.99
epoch:6,Train Loss: 0.0011,Train Acc: 0.99,Test Loss: 0.0013,Test Acc: 0.99
epoch:7,Train Loss: 0.0010,Train Acc: 0.99,Test Loss: 0.0012,Test Acc: 0.99


In [ ]:
time_end = time.time()
time_c= time_end - time_start
print('time cost', time_c, 's')

In [ ]:
# ploting training and testing loss
plt.plot(train_losses,label='training loss')
plt.plot(test_losses,label='testing loss')
plt.title("Training and Testing Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc = "upper right")
plt.show()

In [ ]:
# ploting training and testing acc
plt.plot(train_acces,label='training accuracy')
plt.plot(test_acces,label='testing accuracy')
plt.title("Training and Testing Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
# cnn.eval()
# x1 = []
# x2 = []
# y_true = []
# y_pred = []

# with torch.no_grad():
#     for data in val_dl:
#         val_x,val_y = data
# #         print(val_x)
# #         print(val_y)
#         val_y = val_y.resize(batch)
#         val_x = val_x.to(torch.float)

#         outputs = cnn(val_x)
#         outputs = outputs.argmax(1)
#         val_x = val_x.cpu()
        
#         x1.append(val_x[outputs == 1].squeeze()) # seismic event
#         x2.append(val_x[outputs == 0].squeeze()) # non seismic event
#         y_pred.append(outputs)  # classfied values
#         y_true.append(val_y)    # true values
        
#     x1 = np.vstack(x1)
#     x2 = np.vstack(x2)    
# #     y_pred = np.vstack(y_pred)
# #     y_true = np.vstack(y_true)
    
#     print(x1.shape)
#     print(x2.shape)
#     print(type(y_pred))
#     print(type(y_true))   

In [ ]:
# # plotting the classification results 
# tt = np.linspace(0, base_size, num = base_size)
# print(tt.shape)
# print(x1.shape)
# print(x2.shape)

# # before event
# plt.plot(tt, x1[0,:], color = 'skyblue',label='Seismic event', linestyle='--')   
# plt.plot(tt, x1[50,:],color = 'skyblue' , linestyle='--')            
# plt.plot(tt, x1[99,:],color = 'skyblue', linestyle='--')

# # after event 
# plt.plot(tt, x2[0,:], color = 'maroon',label='Non-Seismic event')   
# plt.plot(tt, x2[50,:], color = 'maroon')             
# plt.plot(tt, x2[99,:], color = 'maroon') 

# plt.legend(loc= "upper left")
# plt.title("Seismograms Classified")
# plt.style.use('seaborn')
# plt.xlabel("Time")
# plt.ylabel("Frequncy")
# plt.show()

In [ ]:

# # """classified results"""
# cnn.eval()
# test_input = test_asd.st[1]
# all_test_x = []
# all_predict = []
# for i in range(0, len(test_input), base_size):
#     temp_input = test_input[i:i + base_size]
#     if len(temp_input) != base_size:
#         continue
#     all_test_x.append(temp_input)
#     temp_input = torch.from_numpy(temp_input)
#     temp_input = temp_input.view(1, base_size, 1)

#     if torch.cuda.is_available():
#         temp_input = temp_input.cuda()
#     temp_input = temp_input.to(torch.float)
#     outputs = cnn(temp_input)
#     outputs = outputs.argmax(1)
#     all_predict.append(torch.flatten(torch.tile(outputs.cpu().view(-1, 1), (1, base_size))))
# all_test_x = np.concatenate(all_test_x)
# all_predict = np.concatenate(all_predict)

# x_data = list(all_predict)
# y_data = list(all_test_x)
# color_list = []

# flag = False
# for i, predict in enumerate(x_data):

#     if flag == True and predict == 0:
#         end = i
#         color_list.append(opts.MarkAreaItem(name="1", x=(start, end)), )
#     elif flag == False and predict == 1:
#         start = i
#     if predict == 1:
#         flag = True
#     else:
#         flag = False
#     if i == len(x_data) - 1:
#         color_list.append(opts.MarkAreaItem(name="1", x=(start, i + 1)), )

# x_data = [str(i) for i in range(len(x_data))]

# line = (
#     Line()
#         .add_xaxis(x_data)
#         .add_yaxis(
#         series_name="",
#         y_axis=y_data,
#         label_opts=opts.LabelOpts(is_show=False),
#                 linestyle_opts=opts.LineStyleOpts(width=2),
#     )
#         .set_global_opts(
#         title_opts=opts.TitleOpts(title="Bar-DataZoom（slider）"),
#                     datazoom_opts=opts.DataZoomOpts(),
#     )
#         .set_series_opts(
#         markarea_opts=opts.MarkAreaOpts(
#             data=color_list
#         )
#     )
# )
# line.render_notebook()
# # line.render("line_style_and_item_style_60.html")


In [ ]:
# tt = np.linspace(0, 720, num = 720)

# plt.subplot(5,1,5)
# plt.subplot(5,1,1)
# plt.plot(tt, st[0].data, color = 'skyblue',label='Seismic event')   
# plt.title("Raw Siesmic Signals")
# plt.subplot(5,1,2)
# plt.plot(tt, st[1].data, color = 'skyblue',label='Seismic event')   

# plt.subplot(5,1,3)
# plt.plot(tt, st[2].data, color = 'skyblue',label='Seismic event') 

# plt.subplot(5,1,4)
# plt.plot(tt, st[3].data, color = 'skyblue',label='Seismic event')   

# plt.subplot(5,1,5)
# plt.plot(tt, st[4].data, color = 'skyblue',label='Seismic event') 

# plt.legend(loc= "upper left")
# plt.style.use('seaborn')
# plt.xlabel("Time")
# plt.ylabel("Frequncy")
# plt.subplots_adjust(left=None, bottom=None, right=None, top=None,
#                 wspace=0, hspace=0.5)
# plt.show()